In [5]:
import pandas as pd
import numpy as np
import requests
import time

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImZlNTY2YWRlYzI5YjQ2ZjE5MTFkM2RkMjlhYWQ0ZDk0IiwiaCI6Im11cm11cjY0In0="
INPUT_FILE = "/content/drive/MyDrive/Edulift/eVED_171101_week_clean.csv"
OUTPUT_FILE = "/content/drive/MyDrive/Edulift/eVED_171101_week_matched.csv"

In [3]:
BATCH_SIZE = 4000
RADIUS = 200

In [6]:
def snap_batch(coords):
    url = "https://api.openrouteservice.org/v2/snap/driving-car"
    headers = {"Authorization": API_KEY, "Content-Type": "application/json"}
    body = {"locations": coords, "radius": RADIUS}
    r = requests.post(url, json=body, headers=headers)
    r.raise_for_status()
    return r.json()

In [7]:
df = pd.read_csv(INPUT_FILE)
coords = df[["lon", "lat"]].values.tolist()

results = []
for i in range(0, len(coords), BATCH_SIZE):
    batch = coords[i:i+BATCH_SIZE]
    print(f"➡️ Processing batch {i}–{i+len(batch)-1}")

    try:
        snapped = snap_batch(batch)
        for j, item in enumerate(snapped.get("locations", [])):
            if item:
                lon, lat = item["location"]
                road = item.get("name")
                results.append([df.iloc[i+j]["VehId"], df.iloc[i+j]["Trip"],
                                df.iloc[i+j]["timestamp"], df.iloc[i+j]["lat"], df.iloc[i+j]["lon"],
                                lat, lon, road])
            else:
                results.append([df.iloc[i+j]["VehId"], df.iloc[i+j]["Trip"],
                                df.iloc[i+j]["timestamp"], df.iloc[i+j]["lat"], df.iloc[i+j]["lon"],
                                None, None, None])
    except Exception as e:
        print(f"⚠️ Error on batch {i}: {e}")
        time.sleep(5)

matched_df = pd.DataFrame(results, columns=[
    "VehId", "Trip", "timestamp", "lat", "lon",
    "matched_lat", "matched_lon", "road_name"
])
matched_df.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Map-matching complete. Saved to {OUTPUT_FILE}")

➡️ Processing batch 0–3999
➡️ Processing batch 4000–7999
➡️ Processing batch 8000–11999
➡️ Processing batch 12000–15999
➡️ Processing batch 16000–19999
➡️ Processing batch 20000–23999
➡️ Processing batch 24000–27999
➡️ Processing batch 28000–31999
➡️ Processing batch 32000–35999
➡️ Processing batch 36000–39999
➡️ Processing batch 40000–43999
➡️ Processing batch 44000–47999
➡️ Processing batch 48000–51999
➡️ Processing batch 52000–55999
➡️ Processing batch 56000–59999
➡️ Processing batch 60000–63999
➡️ Processing batch 64000–67999
➡️ Processing batch 68000–71999
➡️ Processing batch 72000–75999
➡️ Processing batch 76000–79999
➡️ Processing batch 80000–83999
➡️ Processing batch 84000–87999
➡️ Processing batch 88000–91999
➡️ Processing batch 92000–95999
➡️ Processing batch 96000–99999
➡️ Processing batch 100000–103999
➡️ Processing batch 104000–107999
➡️ Processing batch 108000–111999
➡️ Processing batch 112000–115999
➡️ Processing batch 116000–119999
➡️ Processing batch 120000–123999
➡️ P

In [6]:
df_matched = pd.read_csv(OUTPUT_FILE)
df_matched.head()

,VehId,Trip,timestamp,lat,lon,matched_lat,matched_lon,road_name
0,8.0,730.0,1970-01-01 00:00:00.000,42.266570,-83.707059,42.266567,-83.70711,Arlington Boulevard
1,8.0,730.0,1970-01-01 00:00:00.600,42.266570,-83.707059,42.266567,-83.70711,Arlington Boulevard
2,8.0,730.0,1970-01-01 00:00:02.100,42.266570,-83.707059,42.266567,-83.70711,Arlington Boulevard
3,8.0,730.0,1970-01-01 00:00:02.800,42.266959,-83.706973,42.266960,-83.70714,Arlington Boulevard
4,8.0,730.0,1970-01-01 00:00:03.200,42.266959,-83.706973,42.266960,-83.70714,Arlington Boulevard


In [7]:
df_matched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534559 entries, 0 to 534558
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   VehId        534559 non-null  float64
 1   Trip         534559 non-null  float64
 2   timestamp    534559 non-null  object 
 3   lat          534559 non-null  float64
 4   lon          534559 non-null  float64
 5   matched_lat  534550 non-null  float64
 6   matched_lon  534550 non-null  float64
 7   road_name    458535 non-null  object 
dtypes: float64(6), object(2)
memory usage: 32.6+ MB


In [8]:
df_matched['road_name'].isnull().sum()

np.int64(76024)